# MOSI Text Extraction
---

In [1]:
from gensim.test.utils import common_texts, get_tmpfile
from gensim.models import Word2Vec, KeyedVectors
import gensim.models as g
import numpy as np
import pandas as pd
import pickle

In [2]:
#Todo: import from utils
def pad_array(array, size):
    array_qtt = array.shape[0]

    if(array_qtt == size):
        return array

    if(array_qtt < size): #pad with 0

        pad_left_count = int((size - array_qtt) / 2)
        pad_right_count = size - array_qtt - pad_left_count

        pad_left = np.zeros((pad_left_count,) + array.shape[1:])
        pad_right = np.zeros((pad_right_count,) + array.shape[1:])

        result_array = np.concatenate((array, pad_left , pad_right))

    else: #resize
        print('Resize')
        result_array = np.resize(array.mean(axis=0).astype(int),
                                 (size,) + array.shape[1:])
    return result_array

In [3]:
path = '/Users/silvana/Library/Mobile Documents/com~apple~CloudDocs/EURECOM/SemProj/Raw - MOSI Dataset/Transcript/Segmented/'
standard_train_fold=['2iD-tVS8NPw', '8d-gEyoeBzc', 'Qr1Ca94K55A', 'Ci-AH39fi3Y', '8qrpnFRGt2A', 'Bfr499ggo-0', 'QN9ZIUWUXsY', '9T9Hf74oK10', '7JsX8y1ysxY', '1iG0909rllw', 'Oz06ZWiO20M', 'BioHAh1qJAQ', '9c67fiY0wGQ', 'Iu2PFX3z_1s', 'Nzq88NnDkEk', 'Clx4VXItLTE', '9J25DZhivz8', 'Af8D0E4ZXaw', 'TvyZBvOMOTc', 'W8NXH0Djyww', '8OtFthrtaJM', '0h-zjBukYpk', 'Vj1wYRQjB-o', 'GWuJjcEuzt8', 'BI97DNYfe5I', 'PZ-lDQFboO8', '1DmNV9C1hbY', 'OQvJTdtJ2H4', 'I5y0__X72p0', '9qR7uwkblbs', 'G6GlGvlkxAQ', '6_0THN4chvY', 'Njd1F0vZSm4', 'BvYR0L6f2Ig', '03bSnISJMiM', 'Dg_0XKD0Mf4', '5W7Z1C_fDaE', 'VbQk4H8hgr0', 'G-xst2euQUc', 'MLal-t_vJPM', 'BXuRRbG0Ugk', 'LSi-o-IrDMs', 'Jkswaaud0hk', '2WGyTLYerpo', '6Egk_28TtTM', 'Sqr0AcuoNnk', 'POKffnXeBds', '73jzhE8R1TQ', 'OtBXNcAL_lE', 'HEsqda8_d0Q', 'VCslbP0mgZI', 'IumbAb8q2dM']
standard_valid_fold=['WKA5OygbEKI', 'c5xsKMxpXnc', 'atnd_PF-Lbs', 'bvLlb-M3UXU', 'bOL9jKpeJRs', '_dI--eQ6qVU', 'ZAIRrfG22O0', 'X3j2zQgwYgE', 'aiEXnCPZubE', 'ZUXBRvtny7o']
standard_test_fold=['tmZoasNr4rU', 'zhpQhgha_KU', 'lXPQBPVc5Cw', 'iiK8YX8oH1E', 'tStelxIAHjw', 'nzpVDcQ0ywM', 'etzxEpPuc6I', 'cW1FSBF59ik', 'd6hH302o4v8', 'k5Y_838nuGo', 'pLTX3ipuDJI', 'jUzDDGyPkXU', 'f_pcplsH_V0', 'yvsjCA6Y5Fc', 'nbWiPyCm4g0', 'rnaNMUZpvvg', 'wMbj6ajWbic', 'cM3Yna7AavY', 'yDtzw_Y-7RU', 'vyB00TXsimI', 'dq3Nf_lMPnE', 'phBUpBr1hSo', 'd3_k5Xpfmik', 'v0zCBqDeKcE', 'tIrG4oNLFzE', 'fvVhgmXxadc', 'ob23OKe5a9Q', 'cXypl4FnoZo', 'vvZ4IcEtiZc', 'f9O3YtZ2VfI', 'c7UH_rxdZv4']


In [4]:
#load model
m = g.Doc2Vec.load('../apnews_dbow/doc2vec.bin')

#model_parameters
max_utterance = 63

In [5]:
csv_path = '../OpinionLevelSentiment.csv'
labelsDF = pd.read_csv(csv_path, skiprows=0, names=['start', 'end', 'video_id', 'segment', 'score'])

In [6]:
def get_doc_vec(filenames, labelsDF):
    result = []
    count = 1
    
    length = []
    labels = []
    
    for filename in sorted(filenames):
        with open(path + filename + '.annotprocessed', 'r') as file:
            txt = file.read()
        utterances = txt.strip().split('\n')
        utterances = [u[u.find('_DELIM_') + 7:].strip() for u in utterances]\
        
        #utterance quantity in each video
        length.append(len(utterances))

        utdocvec = []
        for i, u in enumerate(utterances):
            utdocvec.append([str(x) for x in m.infer_vector(u.split(" "))])
            
        
        #get labels
        df_segment = labelsDF[labelsDF.video_id == filename].sort_values(by=['segment'])
        scores = df_segment['score'].values
        labels.append(pad_array(scores, max_utterance))
            
        result.append(pad_array(np.array(utdocvec), max_utterance))
        
        
    return np.array(result), length, np.array(labels)

In [7]:
test, test_length, test_label = get_doc_vec(standard_test_fold, labelsDF)
# np.save('mosi_text_test', test)

train, train_length, train_label = get_doc_vec(standard_train_fold + standard_valid_fold, labelsDF)
# np.save('mosi_text_train', train)

### Generating pickle

In [8]:
pickle_path = "../mosi_text_pad_right_sorted.pkl"

data = train, train_label, test, test_label, train.shape[1], train_length, test_length

pickle_out = open(pickle_path, "wb")
pickle.dump(data, pickle_out)
pickle_out.close()

print('Done')

Done
